In [61]:
import sys, os, inspect, logging, importlib

import geopandas as gpd
import pandas as pd
import numpy as np
import osmnx as ox
import networkx as nx

import matplotlib.pyplot as plt

sys.path.append(r'../../GOSTNets/GOSTNets')
import GOSTnet as gn

from shapely.geometry import Point, LineString

## Pre-Processing Step
### The input for the trade flows is an OD Matrix that only has the upper-right diagonal filled out. Therefore the OD Matrix needs to be transposed

In [62]:
# read in OD Matrix
input_folder = r"./input_folder_Feb12_v21_v2032"
# need to make sure the index and the column headers are the same dtype, so when you calculate the 
# transpose matrix you can add it back to the imported matrix
test_input_trade_flows = pd.read_csv(os.path.join(input_folder, "trade_flows_Feb12_v21_v2032.csv"), index_col = 0)

In [63]:
test_input_trade_flows.columns = test_input_trade_flows.columns.astype(int)

In [64]:
test_input_trade_flows_transposed = test_input_trade_flows.transpose()

In [65]:
test_input_trade_flows_mirrored = test_input_trade_flows + test_input_trade_flows_transposed
test_input_trade_flows_mirrored

4        8       10       22       25       33       36       48  \
4         0        0       0        0        0   950000   687606        0   
8         0        0       0        0        0  1000000        0        0   
10        0        0       0        0        0   100000        0        0   
22        0        0       0        0   130085    50000  1737602        0   
25        0        0       0   130085        0   175000  1392500        0   
33   950000  1000000  100000    50000   175000        0    50000        0   
36   687606        0       0  1737602  1392500    50000        0        0   
48        0        0       0        0        0        0        0        0   
54        0        0       0   130085        0   175000  1392500        0   
60        0        0       0   130085        0   175000  1566410        0   
63   865349        0       0    25000    53563  5000000    47944        0   
69     9906    21668       0    50000   388468   100000   451054        0   
89   250000   100000   50000    50000   242684        0    18214    25000   
90     5000    10000   25000     9000     2000   500000        0    50000   
91  1585286    50000       0   136554   107776   250000   687606   200000   
92   120360   100000   50000    60909    12268  3520458     5000  2000000   
93  1000000   500000   10000   100000   100000   500000   100000        0   

         54       60       63       69       89       90       91       92  \
4         0        0   865349     9906   250000     5000  1585286   120360   
8         0        0        0    21668   100000    10000    50000   100000   
10        0        0        0        0    50000    25000        0    50000   
22   130085   130085    25000    50000    50000     9000   136554    60909   
25        0        0    53563   388468   242684     2000   107776    12268   
33   175000   175000  5000000   100000        0   500000   250000  3520458   
36  1392500  1566410    47944   451054    18214        0   687606     5000   
48        0        0        0        0    25000    50000   200000  2000000   
54        0        0    53563   388468   242684     2000   107776    12268   
60        0        0    53563   388468   242684     2000   107776    12268   
63    53563    53563        0  1022230  3000000  1000000   100000  7697961   
69   388468   388468  1022230        0   100000    10000   250000    50000   
89   242684   242684  3000000   100000        0   400000   828810  2620458   
90     2000     2000  1000000    10000   400000        0        0        0   
91   107776   107776   100000   250000   828810        0        0        0   
92    12268    12268  7697961    50000  2620458        0        0        0   
93  1000000  1000000    50000    50000  1000000        0        0        0   

         93  
4   1000000  
8    500000  
10    10000  
22   100000  
25   100000  
33   500000  
36   100000  
48        0  
54  1000000  
60  1000000  
63    50000  
69    50000  
89  1000000  
90        0  
91        0  
92        0  
93        0

## Defining Inputs and Outputs

In [66]:
scenario_var = "_Feb12_v21_v2032"
#cost_attr = 'cost'

# Read in and process input data
input_folder = r"./input_folder_Feb12_v21_v2032"
geo_data = r"./geographic_data_Feb12_v12"

OD_df = pd.read_csv(os.path.join(input_folder, "asia_railways_output_OD_matrix{}.csv".format(scenario_var)), index_col = 0)

G_pickle  = os.path.join(input_folder, "asia_railways_ver3{}.pickle".format(scenario_var))
G = nx.read_gpickle(G_pickle)

nodes_geography = os.path.join(geo_data, "rail_nodes_Feb12_v12.csv")
nodes_data = pd.read_csv(nodes_geography, dtype = {'X': np.float64, 'Y': np.float64})
nodes_data['Point'] = [Point(x, y) for x, y in zip(nodes_data['X'],nodes_data['Y'])]
nodes_data['node_id'] = nodes_data['node_id'].astype(str)

# Output files
output_folder = r"./output_folder{}".format(scenario_var)
complete_volume_map = os.path.join(output_folder, "complete_equal_volume_map_Feb12_v21_v2032.csv")
trade_volume_map = os.path.join(output_folder, "trade_volume_map_Feb21_v10_v2032.csv")

In [67]:
nodes_data[:8]

node_id      name country  border  gauge_change  \
0       1     Aktau     KAZ   False         False   
1       2    Aktobe     KAZ   False         False   
2       3   Aktogal     KAZ   False         False   
3       4    Almaty     KAZ   False         False   
4       5   Andijan     UBZ    True         False   
5       6    Angren     UBZ   False         False   
6       7      Arys     KAZ   False         False   
7       8  Ashgabat     TKM   False         False   

   if_border_is_an_origin_or_destination      Name          X          Y  \
0                                  False     Aktau  51.194236  43.650086   
1                                  False    Aktobe  57.166930  50.283883   
2                                  False   Aktogal  79.922600  46.851951   
3                                  False    Almaty  76.857735  43.229664   
4                                  False   Andijan  72.284590  40.814750   
5                                  False    Angren  70.074230  41.008453   
6                                  False      Arys  68.813051  42.433659   
7                                  False  Ashgabat  58.316298  37.943921   

                                  Point  
0        POINT (51.19423615 43.6500856)  
1       POINT (57.16692971 50.28388335)  
2  POINT (79.92260014999999 46.8519507)  
3       POINT (76.85773523 43.22966422)  
4  POINT (72.28458996000001 40.8147501)  
5       POINT (70.07422952 41.00845288)  
6        POINT (68.8130505 42.43365853)  
7       POINT (58.31629783 37.94392081)

In [68]:
OD_df

38             69             70             36  \
38        0.171170  100000.171170  100012.021170  100168.996170   
69   100000.171170       0.171170  100206.001170     168.996170   
70   100012.021170  100206.001170       0.171170  100037.176170   
36   100168.996170     168.996170  100037.176170       0.171170   
5    100130.181170     130.181170  100075.991170      38.986170   
..             ...            ...            ...            ...   
90   500337.101170  400337.101170  500331.811170  400294.806170   
91   300352.591171  200352.591171  300347.301171  200310.296171   
93   300352.591171  200352.591171  300347.301171  200310.296171   
18b  100298.611170     298.611170  100359.661170     322.656170   
68b  100156.541171     156.541171  100151.251171     114.246171   

                 5             25             35             54  \
38   100130.181170  100115.641170  100118.921170  100115.011170   
69      130.181170     115.641170     118.921170     115.011170   
70   100075.991170  100090.531170  100105.811170  100109.721170   
36       38.986170      53.526170      68.806170      72.716170   
5         0.171170      14.711170      29.991170      33.901170   
..             ...            ...            ...            ...   
90   400255.991170  400241.451170  400226.171170  400222.261170   
91   200271.481171  200256.941171  200241.661171  200237.751171   
93   200271.481171  200256.941171  200241.661171  200237.751171   
18b     283.841170     269.301170     272.581170     268.671170   
68b      75.431171      60.891171      45.611171      41.701171   

                37             65  ...             80             81  \
38   100121.451170  100133.511170  ...  400328.761171  400402.591171   
69      121.451170     133.511170  ...  300328.761171  300402.591171   
70   100116.161170  100128.221170  ...  400323.471171  400397.301171   
36       79.156170      91.216170  ...  300286.466171  300360.296171   
5        40.341170      52.401170  ...  300247.651171  300321.481171   
..             ...            ...  ...            ...            ...   
90   400215.821170  400203.761170  ...  200050.171170  100025.171170   
91   200234.851171  200246.911171  ...  400175.171170  300150.171170   
93   200234.851171  200246.911171  ...  200075.171170  100050.171170   
18b     275.111170     287.171170  ...  300473.501170  300547.331170   
68b      38.801171      50.861171  ...  300172.391170  300246.221170   

                89             94             92             90  \
38   300187.211170  400214.711170  500242.211170  500337.101170   
69   200187.211170  300214.711170  400242.211170  400337.101170   
70   300181.921170  400209.421170  500236.921170  500331.811170   
36   200144.916170  300172.416170  400199.916170  400294.806170   
5    200106.101170  300133.601170  400161.101170  400255.991170   
..             ...            ...            ...            ...   
90   200150.061170  100122.561170      95.061170       0.171170   
91   400300.611171  500297.561170  400270.061170  400175.171170   
93   400225.061170  300197.561170  200170.061170  200075.171170   
18b  200340.871170  300368.371170  400395.871170  400490.761170   
68b  200104.561171  300132.061171  400159.561171  400254.451171   

                91             93            18b            68b  
38   300352.591171  300352.591171  100298.611170  100156.541171  
69   200352.591171  200352.591171     298.611170     156.541171  
70   300347.301171  300347.301171  100359.661170  100151.251171  
36   200310.296171  200310.296171     322.656170     114.246171  
5    200271.481171  200271.481171     283.841170      75.431171  
..             ...            ...            ...            ...  
90   400175.171170  200075.171170  400490.761170  400254.451171  
91        0.171170  200100.171170  200497.331170  200196.221170  
93   200100.171170       0.171170  200497.331170  200196.221170  
18b  200497.331170  200497.331170       0.171170     310.201171  
68b  20019

# Map network volume based on ALL least-cost trips

In [69]:
def tabulate_edges(route):
    edge_table = []
    for idx in range(0, len(route) - 1):
        edge_table.append([route[idx], route[idx+1]])
    return(edge_table)


In [70]:
complete_edges = []

count = 0

# This will iterate through all of the values in the OD Matrix, therefore you will see edge being counted 
# in both directions, for example if there is an edge of node 1 to node 2, then there will be an edge 
# corresponding to of node 2 to node 1

for origin, row in OD_df.iterrows():    
    for destination, value in row.items():
        try:
            origin = int(origin)
        except:
            pass
        try:
            destination = int(destination)
        except:
            pass
        
        count = count + 1
        #print(value)
        route = nx.dijkstra_path(G, origin, destination, weight = 'cost')
        # print(route)
        edge_table = tabulate_edges(route)
        print(edge_table)
        complete_edges = complete_edges + edge_table 

[]
[[38, 69]]
[[38, 70]]
[[38, 69], [69, 40], [40, 4], [4, 17], [17, 7], [7, 60], [60, 6], [6, 50], [50, 25], [25, 5], [5, 36]]
[[38, 69], [69, 40], [40, 4], [4, 17], [17, 7], [7, 60], [60, 6], [6, 50], [50, 25], [25, 5]]
[[38, 69], [69, 40], [40, 4], [4, 17], [17, 7], [7, 60], [60, 6], [6, 50], [50, 25]]
[[38, 69], [69, 40], [40, 4], [4, 17], [17, 7], [7, 60], [60, 54], [54, 35]]
[[38, 69], [69, 40], [40, 4], [4, 17], [17, 7], [7, 60], [60, 54]]
[[38, 69], [69, 40], [40, 4], [4, 17], [17, 7], [7, 60], [60, 54], [54, 37]]
[[38, 69], [69, 40], [40, 4], [4, 17], [17, 7], [7, 60], [60, 54], [54, 37], [37, 65]]
[[38, 69], [69, 40], [40, 4], [4, 17], [17, 7], [7, 60], [60, 54], [54, 37], [37, 65], [65, 31]]
[[38, 69], [69, 40], [40, 4], [4, 17], [17, 7], [7, 60], [60, 54], [54, 39]]
[[38, 69], [69, 40], [40, 4], [4, 17], [17, 7], [7, 60], [60, 54], [54, 39], [39, 68]]
[[38, 69], [69, 40], [40, 4], [4, 17], [17, 7], [7, 60], [60, 54], [54, 39], [39, 68], [68, '68b'], ['68b', 43]]
[[38, 69], 

[[39, 37], [37, 65], [65, 31], [31, 42], [42, 59]]
[[39, 68], [68, '68b'], ['68b', 43], [43, 58], [58, 55], [55, 45], [45, 66], [66, 83], [83, 33]]
[[39, 68], [68, '68b'], ['68b', 43], [43, 58], [58, 55], [55, 45], [45, 66], [66, 83]]
[[39, 54], [54, 60], [60, 7], [7, 17], [17, 4], [4, 40]]
[[39, 54], [54, 60], [60, 7], [7, 17], [17, 4]]
[[39, 54], [54, 60], [60, 7], [7, 17]]
[[39, 54], [54, 60], [60, 7]]
[[39, 54], [54, 60], [60, 7], [7, 53]]
[[39, 54], [54, 60], [60, 7], [7, 53], [53, 16]]
[[39, 54], [54, 60], [60, 7], [7, 53], [53, 16], [16, 1]]
[[39, 54], [54, 60], [60, 7], [7, 53], [53, 16], [16, 1], [1, 71]]
[[39, 54], [54, 60], [60, 7], [7, 53], [53, 16], [16, 1], [1, 71], [71, 86]]
[[39, 54], [54, 60], [60, 7], [7, 53], [53, 16], [16, 1], [1, 71], [71, 86], [86, 30]]
[[39, 54], [54, 60], [60, 7], [7, 53], [53, 16], [16, 1], [1, 71], [71, 86], [86, 30], [30, 14]]
[[39, 54], [54, 60], [60, 7], [7, 53], [53, 16], [16, 1], [1, 71], [71, 86], [86, 30], [30, 14], [14, 9]]
[[39, 68], 

[[45, 26], [26, 57], [57, 28], [28, 85], [85, 9], [9, 14], [14, 30], [30, 86], [86, 71], [71, 1], [1, 16], [16, 53], [53, 78], [78, 77]]
[[45, 26], [26, 57], [57, 28], [28, 85], [85, 9], [9, 14], [14, 30], [30, 86], [86, 71], [71, 1], [1, 16], [16, 53], [53, 78]]
[[45, 63], [63, 52]]
[[45, 63], [63, 52], [52, 72]]
[[45, 63], [63, 52], [52, 72], [72, 61]]
[[45, 55], [55, 58], [58, 43], [43, 8]]
[[45, 55], [55, 58], [58, 43], [43, 8], [8, 67]]
[[45, 55], [55, 58], [58, 43], [43, '68b'], ['68b', 68], [68, 39], [39, 54], [54, 60]]
[[45, 63]]
[[45, 55], [55, 58], [58, 43], [43, '68b'], ['68b', 68], [68, 39], [39, 54], [54, 35], [35, 25], [25, 50], [50, 6]]
[[45, 55], [55, 58], [58, 43], [43, '68b'], ['68b', 68], [68, 39], [39, 54], [54, 35], [35, 25], [25, 50]]
[[45, 55], [55, 58], [58, 43], [43, 8], [8, 67], [67, 10], [10, 62], [62, 51]]
[[45, 63], [63, 52], [52, 72], [72, 61], [61, 34], [34, 32], [32, 41], [41, 47]]
[[45, 63], [63, 52], [52, 72], [72, 61], [61, 34], [34, 32], [32, 41]]
[[

[[42, 89], [89, 94], [94, 92], [92, 90]]
[[42, 31], [31, 65], [65, 37], [37, 39], [39, 68], [68, '68b'], ['68b', 43], [43, 58], [58, 55], [55, 45], [45, 66], [66, 12], [12, 76], [76, 20], [20, 91]]
[[42, 31], [31, 65], [65, 37], [37, 39], [39, 68], [68, '68b'], ['68b', 43], [43, 58], [58, 55], [55, 45], [45, 66], [66, 12], [12, 76], [76, 20], [20, 93]]
[[42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 7], [7, 53], [53, 2], [2, 87], [87, 48], [48, 56], [56, 29], [29, 74], [74, 27], [27, '18b']]
[[42, 31], [31, 65], [65, 37], [37, 39], [39, 68], [68, '68b']]
[[59, 42], [42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 7], [7, 17], [17, 4], [4, 40], [40, 69], [69, 38]]
[[59, 42], [42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 7], [7, 17], [17, 4], [4, 40], [40, 69]]
[[59, 42], [42, 31], [31, 65], [65, 37], [37, 54], [54, 35], [35, 25], [25, 5], [5, 36], [36, 70]]
[[59, 42], [42, 31], [31, 65], [65, 37], [37, 54], [54, 35], [35, 25], [25, 5], [5, 36]]
[[59, 42], [42, 31]

[[17, 4], [4, 40], [40, 69], [69, 38]]
[[17, 4], [4, 40], [40, 69]]
[[17, 7], [7, 60], [60, 6], [6, 50], [50, 25], [25, 5], [5, 36], [36, 70]]
[[17, 7], [7, 60], [60, 6], [6, 50], [50, 25], [25, 5], [5, 36]]
[[17, 7], [7, 60], [60, 6], [6, 50], [50, 25], [25, 5]]
[[17, 7], [7, 60], [60, 6], [6, 50], [50, 25]]
[[17, 7], [7, 60], [60, 54], [54, 35]]
[[17, 7], [7, 60], [60, 54]]
[[17, 7], [7, 60], [60, 54], [54, 37]]
[[17, 7], [7, 60], [60, 54], [54, 37], [37, 65]]
[[17, 7], [7, 60], [60, 54], [54, 37], [37, 65], [65, 31]]
[[17, 7], [7, 60], [60, 54], [54, 39]]
[[17, 7], [7, 60], [60, 54], [54, 39], [39, 68]]
[[17, 7], [7, 60], [60, 54], [54, 39], [39, 68], [68, '68b'], ['68b', 43]]
[[17, 7], [7, 60], [60, 54], [54, 39], [39, 68], [68, '68b'], ['68b', 43], [43, 58]]
[[17, 7], [7, 60], [60, 54], [54, 39], [39, 68], [68, '68b'], ['68b', 43], [43, 58], [58, 55]]
[[17, 7], [7, 60], [60, 54], [54, 39], [39, 68], [68, '68b'], ['68b', 43], [43, 58], [58, 55], [55, 45]]
[[17, 7], [7, 60], [60, 54

...

[[80, 81], [81, 90], [90, 92]]
[[80, 81], [81, 90]]
[[80, 81], [81, 93], [93, 20], [20, 91]]
[[80, 81], [81, 93]]
[[80, 79], [79, 33], [33, 83], [83, 66], [66, 45], [45, 26], [26, 57], [57, 28], [28, 85], [85, 9], [9, 14], [14, 30], [30, 86], [86, 71], [71, 1], [1, 16], [16, 53], [53, 2], [2, 87], [87, 48], [48, 56], [56, 29], [29, 74], [74, 27], [27, '18b']]
[[80, 79], [79, 33], [33, 83], [83, 66], [66, 45], [45, 55], [55, 58], [58, 43], [43, '68b']]
[[81, 93], [93, 20], [20, 76], [76, 12], [12, 66], [66, 45], [45, 55], [55, 58], [58, 43], [43, '68b'], ['68b', 68], [68, 39], [39, 54], [54, 60], [60, 7], [7, 17], [17, 4], [4, 40], [40, 69], [69, 38]]
[[81, 93], [93, 20], [20, 76], [76, 12], [12, 66], [66, 45], [45, 55], [55, 58], [58, 43], [43, '68b'], ['68b', 68], [68, 39], [39, 54], [54, 60], [60, 7], [7, 17], [17, 4], [4, 40], [40, 69]]
[[81, 93], [93, 20], [20, 76], [76, 12], [12, 66], [66, 45], [45, 55], [55, 58], [58, 43], [43, '68b'], ['68b', 68], [68, 39], [39, 54], [54, 35], [

[[94, 89], [89, 42], [42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 7], [7, 53], [53, 16], [16, 1], [1, 71], [71, 86]]
[[94, 89], [89, 42], [42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 7], [7, 53], [53, 16], [16, 1], [1, 71], [71, 86], [86, 30]]
[[94, 89], [89, 42], [42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 7], [7, 53], [53, 16], [16, 1], [1, 71], [71, 86], [86, 30], [30, 14]]
[[94, 89], [89, 42], [42, 31], [31, 65], [65, 37], [37, 54], [54, 60], [60, 7], [7, 53], [53, 16], [16, 1], [1, 71], [71, 86], [86, 30], [30, 14], [14, 9]]
[[94, 89], [89, 42], [42, 31], [31, 65], [65, 37], [37, 39], [39, 68], [68, '68b'], ['68b', 43], [43, 58], [58, 55], [55, 45], [45, 26], [26, 57], [57, 28], [28, 85]]
[[94, 89], [89, 42], [42, 31], [31, 65], [65, 37], [37, 39], [39, 68], [68, '68b'], ['68b', 43], [43, 58], [58, 55], [55, 45], [45, 26], [26, 57], [57, 28]]
[[94, 89], [89, 42], [42, 31], [31, 65], [65, 37], [37, 39], [39, 68], [68, '68b'], ['68b', 43], [43, 58], [5

[[90, 81], [81, 93], [93, 20], [20, 76], [76, 12], [12, 66], [66, 45], [45, 63], [63, 52]]
[[90, 81], [81, 93], [93, 20], [20, 76], [76, 12], [12, 66], [66, 45], [45, 63], [63, 52], [52, 72]]
[[90, 81], [81, 93], [93, 20], [20, 76], [76, 12], [12, 66], [66, 45], [45, 63], [63, 52], [52, 72], [72, 61]]
[[90, 81], [81, 93], [93, 20], [20, 76], [76, 12], [12, 66], [66, 45], [45, 55], [55, 58], [58, 43], [43, 8]]
[[90, 81], [81, 93], [93, 20], [20, 76], [76, 12], [12, 66], [66, 45], [45, 55], [55, 58], [58, 43], [43, 8], [8, 67]]
[[90, 92], [92, 94], [94, 89], [89, 42], [42, 31], [31, 65], [65, 37], [37, 54], [54, 60]]
[[90, 81], [81, 93], [93, 20], [20, 76], [76, 12], [12, 66], [66, 45], [45, 63]]
[[90, 92], [92, 94], [94, 89], [89, 42], [42, 31], [31, 65], [65, 37], [37, 54], [54, 35], [35, 25], [25, 50], [50, 6]]
[[90, 92], [92, 94], [94, 89], [89, 42], [42, 31], [31, 65], [65, 37], [37, 54], [54, 35], [35, 25], [25, 50]]
[[90, 81], [81, 93], [93, 20], [20, 76], [76, 12], [12, 66], [66,

[[93, 20], [20, 76], [76, 12], [12, 66], [66, 45], [45, 55], [55, 58], [58, 43], [43, '68b'], ['68b', 68], [68, 39], [39, 54], [54, 35], [35, 25], [25, 50]]
[[93, 20], [20, 76], [76, 12], [12, 66], [66, 45], [45, 55], [55, 58], [58, 43], [43, 8], [8, 67], [67, 10], [10, 62], [62, 51]]
[[93, 20], [20, 76], [76, 12], [12, 66], [66, 45], [45, 63], [63, 52], [52, 72], [72, 61], [61, 34], [34, 32], [32, 41], [41, 47]]
[[93, 20], [20, 76], [76, 12], [12, 66], [66, 45], [45, 63], [63, 52], [52, 72], [72, 61], [61, 34], [34, 32], [32, 41]]
[[93, 20], [20, 76], [76, 12], [12, 66], [66, 45], [45, 63], [63, 52], [52, 72], [72, 61], [61, 34], [34, 73]]
[[93, 20], [20, 76], [76, 12], [12, 66], [66, 45], [45, 55], [55, 58], [58, 43], [43, '68b'], ['68b', 68], [68, 39], [39, 37], [37, 22]]
[[93, 20], [20, 76], [76, 12], [12, 66], [66, 45], [45, 55], [55, 58], [58, 43], [43, '68b'], ['68b', 68], [68, 39], [39, 54], [54, 35], [35, 25], [25, 5], [5, 36], [36, 49]]
[[93, 81], [81, 80], [80, 79]]
[[93, 81

[['68b', 43], [43, 58], [58, 55], [55, 45], [45, 63]]
[['68b', 68], [68, 39], [39, 54], [54, 35], [35, 25], [25, 50], [50, 6]]
[['68b', 68], [68, 39], [39, 54], [54, 35], [35, 25], [25, 50]]
[['68b', 43], [43, 8], [8, 67], [67, 10], [10, 62], [62, 51]]
[['68b', 68], [68, 39], [39, 54], [54, 60], [60, 7], [7, 53], [53, 2], [2, 87], [87, 48], [48, 56], [56, 29], [29, 74], [74, 27], [27, '18b'], ['18b', 18], [18, 75], [75, 88], [88, 15], [15, 32], [32, 41], [41, 47]]
[['68b', 68], [68, 39], [39, 54], [54, 60], [60, 7], [7, 53], [53, 2], [2, 87], [87, 48], [48, 56], [56, 29], [29, 74], [74, 27], [27, '18b'], ['18b', 18], [18, 75], [75, 88], [88, 15], [15, 32], [32, 41]]
[['68b', 68], [68, 39], [39, 54], [54, 60], [60, 7], [7, 53], [53, 2], [2, 87], [87, 48], [48, 56], [56, 29], [29, 74], [74, 27], [27, '18b'], ['18b', 18], [18, 73]]
[['68b', 68], [68, 39], [39, 37], [37, 22]]
[['68b', 68], [68, 39], [39, 54], [54, 35], [35, 25], [25, 5], [5, 36], [36, 49]]
[['68b', 43], [43, 58], [58, 55],

In [71]:
count

8100

In [72]:
len(complete_edges)

78878

In [73]:
complete_edges[6]

[17, 7]

In [74]:
complete = pd.DataFrame(complete_edges, columns = ['o', 'd'])
complete['w'] = 1
complete_count = pd.DataFrame(complete.groupby(['o','d']).count())
complete_count.reset_index(inplace=True)
complete_count['o'] = complete_count['o'].astype(str)
complete_count['d'] = complete_count['d'].astype(str)

In [75]:
complete_count = pd.merge(complete_count, nodes_data.loc[:,['node_id','Point']], left_on = 'o', right_on='node_id')
complete_count = pd.merge(complete_count, nodes_data.loc[:,['node_id','Point']], left_on = 'd', right_on='node_id')
complete_count['geometry'] = [LineString([x,y]) for x, y in zip(complete_count['Point_x'], complete_count["Point_y"])]
complete_count = complete_count.loc[:,['o','d','w','geometry']]

In [76]:
#complete_count.loc[complete_count['o']=='62']

In [77]:
#complete_count.loc[complete_count['d']=='62']

In [78]:
complete_count

o   d    w                                           geometry
0     1  10  308  LINESTRING (51.19423615 43.6500856, 49.8663731...
1    62  10  381  LINESTRING (44.82710578 41.71513710000001, 49....
2    67  10  244  LINESTRING (52.96318620000001 40.02016264, 49....
3     1  16  905  LINESTRING (51.19423615 43.6500856, 55.1818477...
4    53  16  926  LINESTRING (60.31394877 47.04978092, 55.181847...
..   ..  ..  ...                                                ...
201  92  90   71  LINESTRING (71.56378993 34.10942691, 67.001097...
202  89  94  215  LINESTRING (69.19380137 34.50670823, 71.111711...
203  92  94  142  LINESTRING (71.56378993 34.10942691, 71.111711...
204  90  92   71  LINESTRING (67.00109753 24.89698664, 71.563789...
205  94  92  142  LINESTRING (71.11171118999999 34.19162304, 71....

[206 rows x 4 columns]

In [79]:
complete_count.to_csv(complete_volume_map)

# Map specific routes with trade flows

In [80]:
# Instead I will be parsing through the trade flows DataFrame (Matrix)
complete_edges_trade = []

for idx in test_input_trade_flows_mirrored.iterrows():
    #print(idx[0])
    row = idx[0]
    #print(idx[1])
    for col,value in idx[1].items():
        #print(col)
        print(row,col,value)
        route = nx.dijkstra_path(G, row, col, weight = 'cost')
        if origin == 91:
            print(origin,destination,route)
        edge_table = tabulate_edges(route)
        [x.append(value) for x in edge_table]
        complete_edges_trade = complete_edges_trade + edge_table 

4 4 0
4 8 0
4 10 0
4 22 0
4 25 0
4 33 950000
4 36 687606
4 48 0
4 54 0
4 60 0
4 63 865349
4 69 9906
4 89 250000
4 90 5000
4 91 1585286
4 92 120360
4 93 1000000
8 4 0
8 8 0
8 10 0
8 22 0
8 25 0
8 33 1000000
8 36 0
8 48 0
8 54 0
8 60 0
8 63 0
8 69 21668
8 89 100000
8 90 10000
8 91 50000
8 92 100000
8 93 500000
10 4 0
10 8 0
10 10 0
10 22 0
10 25 0
10 33 100000
10 36 0
10 48 0
10 54 0
10 60 0
10 63 0
10 69 0
10 89 50000
10 90 25000
10 91 0
10 92 50000
10 93 10000
22 4 0
22 8 0
22 10 0
22 22 0
22 25 130085
22 33 50000
22 36 1737602
22 48 0
22 54 130085
22 60 130085
22 63 25000
22 69 50000
22 89 50000
22 90 9000
22 91 136554
22 92 60909
22 93 100000
25 4 0
25 8 0
25 10 0
25 22 130085
25 25 0
25 33 175000
25 36 1392500
25 48 0
25 54 0
25 60 0
25 63 53563
25 69 388468
25 89 242684
25 90 2000
25 91 107776
25 92 12268
25 93 100000
33 4 950000
33 8 1000000
33 10 100000
33 22 50000
33 25 175000
33 33 0
33 36 50000
33 48 0
33 54 175000
33 60 175000
33 63 5000000
33 69 100000
33 89 0
33 90 500000
3

In [81]:
len(complete_edges_trade)

2388

In [82]:
complete_edges_trade

[[4, 17, 0],
 [17, 7, 0],
 [7, 60, 0],
 [60, 54, 0],
 [54, 39, 0],
 [39, 68, 0],
 [68, '68b', 0],
 ['68b', 43, 0],
 [43, 8, 0],
 [4, 17, 0],
 [17, 7, 0],
 [7, 53, 0],
 [53, 16, 0],
 [16, 1, 0],
 [1, 10, 0],
 [4, 17, 0],
 [17, 7, 0],
 [7, 60, 0],
 [60, 54, 0],
 [54, 37, 0],
 [37, 22, 0],
 [4, 17, 0],
 [17, 7, 0],
 [7, 60, 0],
 [60, 6, 0],
 [6, 50, 0],
 [50, 25, 0],
 [4, 17, 950000],
 [17, 7, 950000],
 [7, 60, 950000],
 [60, 54, 950000],
 [54, 39, 950000],
 [39, 68, 950000],
 [68, '68b', 950000],
 ['68b', 43, 950000],
 [43, 58, 950000],
 [58, 55, 950000],
 [55, 45, 950000],
 [45, 66, 950000],
 [66, 83, 950000],
 [83, 33, 950000],
 [4, 17, 687606],
 [17, 7, 687606],
 [7, 60, 687606],
 [60, 6, 687606],
 [6, 50, 687606],
 [50, 25, 687606],
 [25, 5, 687606],
 [5, 36, 687606],
 [4, 17, 0],
 [17, 7, 0],
 [7, 53, 0],
 [53, 2, 0],
 [2, 87, 0],
 [87, 48, 0],
 [4, 17, 0],
 [17, 7, 0],
 [7, 60, 0],
 [60, 54, 0],
 [4, 17, 0],
 [17, 7, 0],
 [7, 60, 0],
 [4, 17, 865349],
 [17, 7, 865349],
 [7, 60, 865

In [83]:
complete = pd.DataFrame(complete_edges_trade, columns=['o', 'd', 'w'])
complete_count = pd.DataFrame(complete.groupby(['o','d']).sum())
complete_count.reset_index(inplace = True)
complete_count['o'] = complete_count['o'].astype(str)
complete_count['d'] = complete_count['d'].astype(str)

In [84]:
complete_count

o   d         w
0      1  10         0
1      1  16    200000
2      1  71    200000
3      2  53   2275000
4      2  87   2275000
..   ...  ..       ...
127   93  81   2035000
128   94  89  14221492
129   94  92  14221492
130  68b  43  21977425
131  68b  68  21977425

[132 rows x 3 columns]

In [85]:
complete_count = pd.merge(complete_count, nodes_data.loc[:,['node_id','Point']], left_on = 'o', right_on = 'node_id')
complete_count = pd.merge(complete_count, nodes_data.loc[:,['node_id','Point']], left_on = 'd', right_on = 'node_id')
complete_count['geometry'] = [LineString([x,y]) for x, y in zip(complete_count['Point_x'], complete_count["Point_y"])]
complete_count = complete_count.loc[:,['o','d','w','geometry']]

In [86]:
complete_count.loc[complete_count['o'] == '17']

o  d        w                                           geometry
45  17  4  8733957  LINESTRING (73.91684012 43.65288192, 76.857735...
47  17  7  8733957  LINESTRING (73.91684012 43.65288192, 68.813050...

In [87]:
complete_count.loc[complete_count['d'] == '17']

o   d        w                                           geometry
11  4  17  8733957  LINESTRING (76.85773523 43.22966422, 73.916840...
12  7  17  8733957  LINESTRING (68.8130505 42.43365853, 73.9168401...

In [88]:
complete_count

o   d         w                                           geometry
0     1  10         0  LINESTRING (51.19423615 43.6500856, 49.8663731...
1    67  10    235000  LINESTRING (52.96318620000001 40.02016264, 49....
2     1  16    200000  LINESTRING (51.19423615 43.6500856, 55.1818477...
3    53  16    200000  LINESTRING (60.31394877 47.04978092, 55.181847...
4     1  71    200000  LINESTRING (51.19423615 43.6500856, 52.8616865...
..   ..  ..       ...                                                ...
127  87  48   2275000  LINESTRING (56.20262023 50.84645319, 55.131690...
128  89  94  14221492  LINESTRING (69.19380137 34.50670823, 71.111711...
129  92  94  14221492  LINESTRING (71.56378993 34.10942691, 71.111711...
130  90  92   5000458  LINESTRING (67.00109753 24.89698664, 71.563789...
131  94  92  14221492  LINESTRING (71.11171118999999 34.19162304, 71....

[132 rows x 4 columns]

In [89]:
complete_count.to_csv(trade_volume_map)